<a href="https://colab.research.google.com/github/Samruddhi-saoji/image-processing-basics/blob/main/image_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image handling

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob

#### Reading and Saving #####
def read(img_path):
   return cv2.imread(img_path, cv2.IMREAD_COLOR)

def read_grayscale(img_path):
   return cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

def save(img, img_path="output.jpeg"):
   cv2.imwrite(img_path, img)


# returns (list of images, list of names)
def list_images(folder_path):
    result = []

    # get list of names of all the images in the folder
    file_names = os.listdir(folder_path)

    # read each image as numpy aaray
    for img_name in file_names:
        # full path of the img = folder_path + image name
        image_path = os.path.join(folder_path, img_name)

        # Read the image and add it to the list
        img = read(image_path) # read as (r,g,b)
        if img is not None:
            result.append(read(image_path))

    return (result, file_names)


#delete all files in a folder
def delete_all_files(folder_path):
    files = glob.glob(os.path.join(folder_path, '*'))
    for file in files:
        if os.path.isfile(file):
            os.remove(file)


#### Updating pixel colour ####
# Update pixel at (r, c) to new_clr
# new_clr = (b, g, r)
def set_pixel(img, pix, new_clr):
    r, c = pix[0], pix[1]
    b_new, g_new, r_new = new_clr[0], new_clr[1], new_clr[2]

    #update the pixel
    img.itemset((r, c, 0), b_new)  # Blue channel
    img.itemset((r, c, 1), g_new)  # Green channel
    img.itemset((r, c, 2), r_new)  # Red channel

    #return updated image
    return img

def set_blue(img, pix, new_blue):
    img.itemset((pix[0], pix[1], 0), new_blue)  # Blue channel
    return img

def set_green(img, pix, new_green):
    img.itemset((pix[0], pix[1], 1), new_green)
    return img

def set_red(img, pix, new_red):
    img.itemset((pix[0], pix[1], 2), new_ref)
    return img

#### Check colour ####
# returns True if given pixel is red
def isRed(pixel):
   b, g, r = pixel
   return r > 200 and g < 100 and b < 100

def isBlue(pixel, thresh=100):
   b, g, r = pixel
   #if (b-g)>=thresh and (b-r)>=thresh: return True
   #return False
   return b > 200 and g < 100 and r < 100

def isGreen(pixel):
   b, g, r = pixel
   #if (g-b)>=75 and (g-r)>=75: return True
   #return False
   return g > 200 and r < 100 and b < 100


# if approx is True, then nearly gray is also ok
def isGray(pixel, approx=False, thresh=20):
   b, g, r = pixel # image is gray if b=g=r

   if not approx:
      if (b == g) and (g == r):
         return True
      else: return False

   else: # approximately gray is ok
      return abs(r - g) < thresh and abs(g - b) < thresh


# if approx is True, then nearly gray is also ok
def isWhite(pixel, approx=True, thresh=200):
   b, g, r = pixel

   if not approx:
      if b!=255 or g!=255 or r!=255: return False
      else: return True

   else: # approximately white is ok
      return r > thresh and g > thresh and b > thresh


def isBlack(pixel, approx=True, thresh=200):
   b, g, r = pixel

   if not approx:
      if b!=0 or g!=0 or r!=0: return False
      else: return True

   else: # approximately black is ok
      return r < thresh and g < thresh and b < thresh


#### basic operations ####
#compress image
  # reduce img from size (r1, c1) to (r2, c2)
  # every (r1/r2)th row and every (c1/c2)th vol of the original img shld be copied to the compressed img
def compress_img(img, r1, c1, r2, c2):
    compressed_img = img[::r1 // r2, ::c1 // c2]
    return compressed_img

# (h_old, w_old) to (h_new, w_new)
def crop(img, r_start, num_rows, c_start, num_cols):
    cropped_img = img[r_start:r_start + num_rows, c_start: c_start + num_cols :]
    return cropped_img

# Processing

## convert to grayscale

In [ ]:
def convert_to_grayscale(img):
   rows, cols, _ = img.shape

   for r in range(rows):
       for c in range(cols):
           blue, green, red = img[r][c]
           gray = (blue + red + green)/3

           # update pixel value
           img[r][c] = (gray, gray, gray)
   return img

## invert colours

In [ ]:
def invert_colours(img):
   rows, cols, _ = img.shape

   for r in range(rows):
       for c in range(cols):
           # current values
           blue, green, red = img[r][c]

           # update value
           img[r][c] = (255-blue, 255-green, 255-red)

   return img

## green screen

In [ ]:
def greenscreen(img, background):
   rows, cols, _ = img.shape

   for r in range(rows):
       for c in range(cols):
           img_pix = img[r][c]
           if is_green(img_pix):
               bg_pix = background[r][c]
               img[r][c] = bg_pix

   return img

## Increase resolution

In [ ]:
def increase_resolution(img):
    rows, cols, _ = img.shape

    new_rows, new_cols = rows*2, cols*2
    new_img = np.zeros((new_rows, new_cols, 3), dtype=img.dtype)

    # copy all rows and cols
    #new_img[0::2][::2] = img
    for r in range(0, new_rows, 2):
        new_img[r][0::2] = img[r//2][:]
    for c in range(0, new_cols, 2):
        new_img[::][c] = img[:][c//2]


    # fill the remaining pixels
    for r in range(1, new_rows-1, 2):
        for c in range(1, new_cols-1,2):
            # if row and col are even, then slip
            #pix = new_img[r][c]
            new_val = (new_img[r-1][c-1] + new_img[r-1][c+1] + new_img[r+1][c-1] + new_img[r+1][c+1])/4
            new_img[r][c] = new_val

    return new_img

## remove background

In [ ]:
def remove_background_transparent(img, bg_colour)

# Testing

In [ ]:
img = read("/content/test.jpg")
result = increase_resolution(img)
save(result)

ValueError: could not broadcast input array from shape (1200,3) into shape (2400,3)